In [ ]:
from datetime import datetime
from typing import NamedTuple, Optional

import pandas as pd

# Zadanie 1 (5 pkt)


Wgraj zawartość pliku orders.csv do ramki danych.
Przyjrzyj się danym, wykorzystując poznane metody (describe, info, head) zanalizuj dane.

Następnie:



1. Zamień order_date na format daty. Upewnij się, że w rezultacie data będzie w formacie: rok-miesiąc-dzień;
2. Sprawdź ile i jakie unikalne wartości znajdują się w kolumnie T-shirt category;
3. Wyczyść kolumnę tshirt_category normalizując nazwy produktów. Upewnij się, że dla każdej kategorii funkcjonuje jedna nazwa, np. zamiast white i wh jest tylko jedna wersja white.
4. Rozdziel kolumnę tshirt category na 3 kolumny: tshirt_type, która będzie mówiła o tym czy jest to koszulka, bluza czy coś innego, tshirt_department, która będzie informowała o tym czy jest męska czy żeńska, i tshirt_colour, która będzie informowała o kolorze. Skorzystaj z metody apply.
5. Znajdź zamówienia złożone między 31 grudnia 2014 a 2 sierpnia 2016.


In [ ]:
orders = pd.read_csv("data/04/orders.csv", sep=",")

# 1.
orders["order_date"] = pd.to_datetime(orders["order_date"])

# 2.
uniq_tshirt_categories = orders["tshirt_category"].unique()
print("BEFORE CLEANING")
print(f"{len(uniq_tshirt_categories)} unique t-shirt categories: {uniq_tshirt_categories}")

# 3.
orders["tshirt_category"] = orders["tshirt_category"].str.replace("Wh Tshirt", "White T-Shirt", regex=False)
orders["tshirt_category"] = orders["tshirt_category"].str.replace("Bl Tshirt", "Black T-Shirt", regex=False)

uniq_tshirt_categories = orders["tshirt_category"].unique()
print("AFTER CLEANING")
print(f"{len(uniq_tshirt_categories)} unique t-shirt categories: {uniq_tshirt_categories}")


# 4.
class TShirtCategory(NamedTuple):
    tshirt_color: Optional[str]
    tshirt_type: Optional[str]
    tshirt_department: Optional[str]

    @staticmethod
    def parse(category: str) -> "TShirtCategory":
        category_parts = category.split(" ")
        match len(category_parts):
            case 3:
                return TShirtCategory(*category_parts)
            case 2 | 1:  # w tych przypadkach 'category' jest od razu typem, a reszty danych brakuje
                return TShirtCategory(None, category, None)
            case _:
                return TShirtCategory(None, None, None)


orders[[*TShirtCategory._fields]] = orders["tshirt_category"].apply(lambda c: pd.Series(TShirtCategory.parse(c)))

# 5.
start_date = datetime(2014, 12, 31)
end_date = datetime(2016, 8, 2)
period = (orders["order_date"] > start_date) & (orders["order_date"] < end_date)
print(f"Orders between {start_date} and {end_date}:")
orders[period].head(99999)

# Zadanie 2 (4 pkt)


Wgraj plik customers.csv do ramki danych.

Następnie:
1. Wypisz kolumny;
2. Porównaj rozmiar ramki z rozmiarem ramki w zadaniu 1;
3. Zmień nazwę kolumny customerID  na customer_id upewniając się, że dane zmiana zostanie zapisana w pamięci dzięki jednemu z argumentów metody do zmiany nazwy kolumny;
4. Połącz ramkę orders z ramką customers po kolumnie customer_id. Z której metody skorzystasz? Zostaw odpowiedź w komentarzu do zadania w kodzie (np. blok komentarzem ‘’’ ‘’’).


In [ ]:
customers = pd.read_csv("data/04/customers.csv", sep=",")
# 1.
print(f"Customer DF columns {customers.columns.tolist()}")

# 2.
orders_rows, orders_cols = orders.shape
customers_rows, customers_cols = customers.shape
print(f"Orders:\n\trows: {orders_rows}\n\tcols: {orders_cols}")
print(f"Customers:\n\trows: {customers_rows}\n\tcols: {customers_cols}")

# 3.
customers.rename({"customerID": "customer_id"}, axis="columns", inplace=True)

# 4.
"""
Left join klientów do zamówień pozwala mieć kompletne dane każdego zamówienia, ale tracimy
w ten sposób dane klientów którzy nic nie zamówili.
Gdybyśmy chcieli nie stracić żadnego rekordu z obu ramek, to inner join też będzie w porządku.
"""
joined_table = pd.merge(orders, customers, on="customer_id", how="left")

joined_table.head(100)

# Zadanie 3 (1 pkt)

Nową ramkę (połączenie orders i customers) zapisz do pliku csv, ale zapisz bez kolumny z indeksem.


In [ ]:
joined_table.to_csv("data/04/zad3_out.csv", index=False)